# Wandb 连接测试工具

本目录包含两个专门用于测试 Wandb 连接性的 Python 脚本，模拟实际工作流程而不是简单的网络ping测试。

## 文件说明

### 1. `test_wandb_connectivity.py` - 完整连接测试
全面的 Wandb 连接测试脚本，包含以下测试项目：

- ✅ **导入和版本检查**: 验证 wandb 包是否正确安装
- ✅ **登录状态检查**: 验证用户是否已登录 wandb
- ✅ **项目初始化**: 测试创建新项目和运行
- ✅ **指标记录**: 模拟训练过程中的指标记录
- ✅ **工件上传**: 测试文件和模型工件上传
- ✅ **运行摘要更新**: 测试更新运行摘要信息
- ✅ **API访问**: 测试通过API读取项目和运行数据
- ✅ **运行结束**: 测试正确结束 wandb 运行

### 2. `test_wandb_quick.py` - 快速连接测试
轻量级的快速测试脚本，适用于日常检查：

- 基本导入测试
- 登录状态检查
- 简单的项目创建和指标记录

## 使用方法

### 环境准备

1. **确保已安装依赖**:
   ```bash
   pip install wandb numpy torch
   ```

2. **配置 Wandb 登录** (如果尚未登录):
   ```bash
   wandb login
   ```
   或者设置环境变量:
   ```bash
   $env:WANDB_API_KEY="your_api_key_here"
   ```

### 运行测试

#### 完整测试 (推荐用于首次设置或故障诊断)
```bash
cd expdemo
python test_wandb_connectivity.py
```

#### 快速测试 (日常检查)
```bash
cd expdemo
python test_wandb_quick.py
```

## 测试结果解读

### 成功情况
- 所有测试项显示 "✅ PASS"
- 最终显示 "🎉 ALL TESTS PASSED"
- 脚本以退出代码 0 结束

### 失败情况分析

1. **导入失败**: 
   - 原因: wandb 未安装或版本不兼容
   - 解决: `pip install --upgrade wandb`

2. **登录失败**:
   - 原因: 未登录或 API key 无效
   - 解决: `wandb login` 重新登录

3. **项目初始化失败**:
   - 原因: 网络连接问题或 wandb 服务不可用
   - 解决: 检查网络连接，尝试使用 VPN

4. **指标记录失败**:
   - 原因: 网络不稳定或上传限制
   - 解决: 检查网络稳定性

5. **API访问失败**:
   - 原因: API 限制或权限问题
   - 解决: 检查账户状态和权限

## 离线模式测试

如果网络连接有问题，可以测试离线模式：

```python
import wandb

# 设置离线模式
wandb.init(mode="offline", project="test-project")
wandb.log({"test": 1})
wandb.finish()

print("离线模式测试完成，数据保存在本地")
```

## 故障排除

### 常见网络问题

1. **防火墙阻拦**:
   - 确保允许 Python 访问网络
   - 检查公司/机构防火墙设置

2. **代理设置**:
   ```bash
   $env:HTTPS_PROXY="http://proxy.company.com:8080"
   $env:HTTP_PROXY="http://proxy.company.com:8080"
   ```

3. **DNS 问题**:
   - 尝试使用不同的 DNS 服务器
   - 检查 hosts 文件

### 测试输出文件

完整测试会生成详细的 JSON 报告:
- 文件名: `wandb_connectivity_test_YYYYMMDD_HHMMSS.json`
- 包含每个测试的详细结果和时间戳
- 可用于故障分析和性能监控

## 与实际项目的对应关系

这些测试脚本模拟了您项目中 `training/trainer.py` 的实际 wandb 使用模式：

- 项目初始化 → `wandb.init()` 在 trainer 构造函数中
- 指标记录 → `_log_metrics()` 方法中的 `wandb.log()`
- 配置管理 → 训练配置传递给 wandb
- 运行管理 → 训练过程的完整生命周期

这确保了测试的实用性和准确性。


In [ ]:
# 设置转发 (该命令在命令行中运行)
!export WANDB_BASE_URL=https://api.bandw.top

In [ ]:
# 配置 wandb 网址 (该命令在代码中运行, 和上面的理论上二选一即可, 不过推荐这个, 写到主代码的最前面) 和 api key
import os
os.environ["WANDB_API_KEY"] = ""
os.environ["WANDB_BASE_URL"] = 'https://api.bandw.top'

## 链接测试

### Full-Test

In [ ]:
#!/usr/bin/env python
"""
Wandb Connectivity Test Script

This script tests wandb connectivity by simulating actual workflow operations
including project creation, logging metrics, and artifact management.
Designed to test connection without simple ping but through actual wandb operations.
"""

import os
import sys
import time
import json
import tempfile
import traceback
from datetime import datetime
from typing import Dict, Any, Optional
import warnings

try:
    import wandb
    import numpy as np
    import torch
except ImportError as e:
    print(f"Missing required packages: {e}")
    print("Please install: pip install wandb numpy torch")
    sys.exit(1)

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

class WandbConnectivityTester:
    """Test wandb connectivity through actual API operations."""
    
    def __init__(self, project_name: str = "connectivity-test"):
        """
        Initialize the tester.
        
        Args:
            project_name: Name of the test project
        """
        self.project_name = project_name
        self.test_run_name = f"test-run-{datetime.now().strftime('%Y%m%d-%H%M%S')}"
        self.test_results = {}
        self.wandb_run = None
        
    def log_test_result(self, test_name: str, success: bool, 
                       details: str = "", duration: float = 0.0):
        """Log individual test results."""
        self.test_results[test_name] = {
            'success': success,
            'details': details,
            'duration': duration,
            'timestamp': datetime.now().isoformat()
        }
        
        status = "✅ PASS" if success else "❌ FAIL"
        print(f"{status} {test_name}: {details} ({duration:.2f}s)")
    
    def test_wandb_import_and_version(self) -> bool:
        """Test if wandb can be imported and check version."""
        start_time = time.time()
        try:
            version = wandb.__version__
            self.log_test_result(
                "Import & Version Check", 
                True, 
                f"wandb v{version} imported successfully",
                time.time() - start_time
            )
            return True
        except Exception as e:
            self.log_test_result(
                "Import & Version Check", 
                False, 
                f"Failed to import wandb: {str(e)}",
                time.time() - start_time
            )
            return False
    
    def test_wandb_login_status(self) -> bool:
        """Test wandb login status."""
        start_time = time.time()
        try:
            # Check if already logged in
            api = wandb.Api()
            user = api.viewer
            username = user.username if hasattr(user, 'username') else 'unknown'
            self.log_test_result(
                "Login Status Check", 
                True, 
                f"Logged in as: {username}",
                time.time() - start_time
            )
            return True
        except Exception as e:
            self.log_test_result(
                "Login Status Check", 
                False, 
                f"Login check failed: {str(e)}",
                time.time() - start_time
            )
            return False
    
    def test_project_initialization(self) -> bool:
        """Test wandb project initialization."""
        start_time = time.time()
        try:
            # Simulate the actual config from your project
            config = {
                'K': 10,
                'lambda_l1': 0.01,
                'learning_rate': 0.001,
                'batch_size': 32,
                'model_name': 'gpt2',
                'max_length': 128,
                'num_epochs': 5,
                'test_mode': True
            }
            
            self.wandb_run = wandb.init(
                project=self.project_name,
                name=self.test_run_name,
                config=config,
                mode='online',  # Force online mode for connectivity test
                reinit=True
            )
            
            self.log_test_result(
                "Project Initialization", 
                True, 
                f"Project '{self.project_name}' initialized, run: {self.test_run_name}",
                time.time() - start_time
            )
            return True
            
        except Exception as e:
            self.log_test_result(
                "Project Initialization", 
                False, 
                f"Failed to initialize project: {str(e)}",
                time.time() - start_time
            )
            return False
    
    def test_metric_logging(self) -> bool:
        """Test logging metrics similar to your training workflow."""
        start_time = time.time()
        try:
            if not self.wandb_run:
                raise Exception("No active wandb run")
            
            # Simulate metrics from your training loop
            test_metrics = []
            
            for epoch in range(3):
                for step in range(5):
                    # Simulate training metrics like in your trainer.py
                    metrics = {
                        'epoch': epoch,
                        'global_step': epoch * 5 + step,
                        'train/ce_loss': np.random.uniform(0.5, 2.0),
                        'train/total_loss': np.random.uniform(0.6, 2.1),
                        'train/kl_divergence': np.random.uniform(0.1, 0.5),
                        'train/sparsity_loss': np.random.uniform(0.01, 0.1),
                        'val/ce_loss': np.random.uniform(0.4, 1.8),
                        'val/total_loss': np.random.uniform(0.5, 1.9),
                        'val/kl_divergence': np.random.uniform(0.05, 0.4),
                        'learning_rate': 0.001 * (0.9 ** epoch)
                    }
                    
                    wandb.log(metrics)
                    test_metrics.append(metrics)
                    time.sleep(0.1)  # Small delay to simulate real training
            
            self.log_test_result(
                "Metric Logging", 
                True, 
                f"Successfully logged {len(test_metrics)} metric entries",
                time.time() - start_time
            )
            return True
            
        except Exception as e:
            self.log_test_result(
                "Metric Logging", 
                False, 
                f"Failed to log metrics: {str(e)}",
                time.time() - start_time
            )
            return False
    
    def test_artifact_upload(self) -> bool:
        """Test artifact upload functionality."""
        start_time = time.time()
        try:
            if not self.wandb_run:
                raise Exception("No active wandb run")
            
            # Create a temporary test file
            with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as f:
                test_data = {
                    'model_config': {'K': 10, 'lambda_l1': 0.01},
                    'test_results': {'accuracy': 0.85, 'loss': 0.32},
                    'timestamp': datetime.now().isoformat()
                }
                json.dump(test_data, f, indent=2)
                temp_file_path = f.name
            
            try:
                # Create and log artifact
                artifact = wandb.Artifact(
                    name="test-model-config",
                    type="config",
                    description="Test artifact for connectivity check"
                )
                artifact.add_file(temp_file_path, name="test_config.json")
                wandb.log_artifact(artifact)
                
                self.log_test_result(
                    "Artifact Upload", 
                    True, 
                    "Successfully uploaded test artifact",
                    time.time() - start_time
                )
                return True
                
            finally:
                # Clean up temp file
                os.unlink(temp_file_path)
                
        except Exception as e:
            self.log_test_result(
                "Artifact Upload", 
                False, 
                f"Failed to upload artifact: {str(e)}",
                time.time() - start_time
            )
            return False
    
    def test_run_summary_update(self) -> bool:
        """Test updating run summary."""
        start_time = time.time()
        try:
            if not self.wandb_run:
                raise Exception("No active wandb run")
            
            # Update run summary with final results
            summary_data = {
                'final_train_loss': 0.45,
                'final_val_loss': 0.52,
                'best_kl_divergence': 0.12,
                'total_training_time': 3600,
                'model_parameters': 124000000,
                'convergence_epoch': 15,
                'test_completed': True
            }
            
            for key, value in summary_data.items():
                wandb.run.summary[key] = value
            
            self.log_test_result(
                "Run Summary Update", 
                True, 
                f"Successfully updated summary with {len(summary_data)} fields",
                time.time() - start_time
            )
            return True
            
        except Exception as e:
            self.log_test_result(
                "Run Summary Update", 
                False, 
                f"Failed to update run summary: {str(e)}",
                time.time() - start_time
            )
            return False
    
    def test_api_access(self) -> bool:
        """Test wandb API access for reading data."""
        start_time = time.time()
        try:
            api = wandb.Api()
            
            # Try to access current user's projects
            projects = list(api.projects(entity=api.default_entity))
            
            # Try to access current run if it exists
            if self.wandb_run:
                run_path = f"{api.default_entity}/{self.project_name}/{self.wandb_run.id}"
                current_run = api.run(run_path)
                run_info = f"Run {current_run.name} (state: {current_run.state})"
            else:
                run_info = "No active run to query"
            
            self.log_test_result(
                "API Access", 
                True, 
                f"Found {len(projects)} projects. {run_info}",
                time.time() - start_time
            )
            return True
            
        except Exception as e:
            self.log_test_result(
                "API Access", 
                False, 
                f"Failed to access API: {str(e)}",
                time.time() - start_time
            )
            return False
    
    def test_wandb_finish(self) -> bool:
        """Test properly finishing wandb run."""
        start_time = time.time()
        try:
            if self.wandb_run:
                wandb.finish()
                self.wandb_run = None
                
                self.log_test_result(
                    "Run Finish", 
                    True, 
                    "Successfully finished wandb run",
                    time.time() - start_time
                )
                return True
            else:
                self.log_test_result(
                    "Run Finish", 
                    False, 
                    "No active run to finish",
                    time.time() - start_time
                )
                return False
                
        except Exception as e:
            self.log_test_result(
                "Run Finish", 
                False, 
                f"Failed to finish run: {str(e)}",
                time.time() - start_time
            )
            return False
    
    def run_all_tests(self) -> Dict[str, Any]:
        """Run all connectivity tests in sequence."""
        print("🔍 Starting Wandb Connectivity Tests...")
        print("=" * 60)
        
        total_start_time = time.time()
        
        # Test sequence - order matters!
        tests = [
            self.test_wandb_import_and_version,
            self.test_wandb_login_status,
            self.test_project_initialization,
            self.test_metric_logging,
            self.test_artifact_upload,
            self.test_run_summary_update,
            self.test_api_access,
            self.test_wandb_finish
        ]
        
        passed_tests = 0
        for test_func in tests:
            try:
                if test_func():
                    passed_tests += 1
            except Exception as e:
                print(f"❌ CRITICAL ERROR in {test_func.__name__}: {str(e)}")
                print(traceback.format_exc())
        
        total_duration = time.time() - total_start_time
        
        # Generate summary
        summary = {
            'total_tests': len(tests),
            'passed_tests': passed_tests,
            'failed_tests': len(tests) - passed_tests,
            'success_rate': (passed_tests / len(tests)) * 100,
            'total_duration': total_duration,
            'test_results': self.test_results,
            'overall_success': passed_tests == len(tests)
        }
        
        print("\n" + "=" * 60)
        print("📊 CONNECTIVITY TEST SUMMARY")
        print("=" * 60)
        print(f"Total Tests: {summary['total_tests']}")
        print(f"Passed: {summary['passed_tests']}")
        print(f"Failed: {summary['failed_tests']}")
        print(f"Success Rate: {summary['success_rate']:.1f}%")
        print(f"Total Duration: {summary['total_duration']:.2f}s")
        
        if summary['overall_success']:
            print("🎉 ALL TESTS PASSED - Wandb connectivity is working properly!")
        else:
            print("⚠️  SOME TESTS FAILED - Check your wandb configuration and network connection")
        
        return summary
    
    def save_results(self, filename: Optional[str] = None) -> str:
        """Save test results to a JSON file."""
        if filename is None:
            filename = f"wandb_connectivity_test_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        
        filepath = os.path.join(os.getcwd(), filename)
        
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(self.test_results, f, indent=2, ensure_ascii=False)
        
        print(f"📁 Test results saved to: {filepath}")
        return filepath


def main():
    """Main function to run wandb connectivity tests."""
    print("🚀 Wandb Connectivity Tester")
    print("Simulating actual workflow operations for comprehensive testing...")
    print()
    
    # Check for environment variables
    if not os.getenv('WANDB_API_KEY'):
        print("⚠️  WANDB_API_KEY environment variable not set.")
        print("   You may need to run 'wandb login' first.")
        print()
    
    # Create tester instance
    tester = WandbConnectivityTester(project_name="minplus-connectivity-test")
    
    try:
        # Run all tests
        results = tester.run_all_tests()
        
        # Save results
        tester.save_results()
        
        # Exit with appropriate code
        exit_code = 0 if results['overall_success'] else 1
        sys.exit(exit_code)
        
    except KeyboardInterrupt:
        print("\n\n⏹️  Test interrupted by user")
        if tester.wandb_run:
            try:
                wandb.finish()
            except:
                pass
        sys.exit(130)
        
    except Exception as e:
        print(f"\n\n💥 CRITICAL ERROR: {str(e)}")
        print(traceback.format_exc())
        if tester.wandb_run:
            try:
                wandb.finish()
            except:
                pass
        sys.exit(1)


if __name__ == "__main__":
    main()


### Quick Test

In [ ]:
#!/usr/bin/env python
"""
Wandb Quick Connectivity Test

A lightweight version of the connectivity test for quick checks.
"""

import sys
import time
import traceback
from datetime import datetime

def quick_wandb_test():
    """Quick wandb connectivity test."""
    print("🔄 Quick Wandb Connectivity Test")
    print("-" * 40)
    
    # Test 1: Import
    try:
        import wandb
        print("✅ Wandb import: SUCCESS")
    except ImportError as e:
        print(f"❌ Wandb import: FAILED - {e}")
        return False
    
    # Test 2: Version check
    try:
        version = wandb.__version__
        print(f"✅ Version: {version}")
    except Exception as e:
        print(f"❌ Version check: FAILED - {e}")
    
    # Test 3: Login status
    try:
        api = wandb.Api(timeout=10)
        user = api.viewer
        if user:
            username = user.get('username', 'unknown')
            print(f"✅ Login status: Logged in as {username}")
        else:
            print("⚠️  Login status: Not logged in")
            return False
    except Exception as e:
        print(f"❌ Login check: FAILED - {e}")
        return False
    
    # Test 4: Quick project init and finish
    try:
        test_project = f"quick-test-{datetime.now().strftime('%Y%m%d-%H%M%S')}"
        run = wandb.init(
            project=test_project,
            name="quick-connectivity-test",
            mode='online',
            config={'test': True}
        )
        
        # Log one test metric
        wandb.log({"test_metric": 1.0})
        
        # Finish
        wandb.finish()
        
        print("✅ Project init/log/finish: SUCCESS")
        return True
        
    except Exception as e:
        print(f"❌ Project operations: FAILED - {e}")
        return False

if __name__ == "__main__":
    try:
        success = quick_wandb_test()
        if success:
            print("\n🎉 Wandb connectivity: WORKING")
            sys.exit(0)
        else:
            print("\n⚠️  Wandb connectivity: ISSUES DETECTED")
            sys.exit(1)
    except Exception as e:
        print(f"\n💥 Critical error: {e}")
        traceback.print_exc()
        sys.exit(1)
